In [ ]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import os
import seaborn as sns
import numpy as np
import tqdm
import datetime

from tensorflow.keras.preprocessing.image import ImageDataGenerator

%load_ext tensorboard

In [ ]:
path = './dataset/train_images/' # Path to directory which contains classes
model = "Xception"
folder_name = "{0}_{1}".format(model, datetime.datetime.now().strftime("%Y-%m-%d-%H:%M:%S"))
checkpoit_folder = './checkpoints/{0}/'.format(folder_name)
tensorbord_folder = "tensorboard/fit/{0}/".format(model)

os.mkdir(tensorbord_folder, 0o755)
os.mkdir(checkpoit_folder, 0o755)

log_dir = tensorbord_folder + datetime.datetime.now().strftime("%Y-%m-%d-%H:%M:%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

checkpoint_filepath = checkpoit_folder + 'epoch-{epoch:02d}-val_loss-{val_loss:.2f}.h5'
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, 
                                                         monitor='val_loss', 
                                                         mode='min')

early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", 
                                                       min_delta=1e-4,
                                                       patience=35,
                                                       verbose=1,
                                                       mode="min")

classes = os.listdir(path) # List of all classes
print('Total number of categories: {0}'.format(len(classes)))

In [ ]:
counts = {}
for c in classes:
    counts[c] = len(os.listdir(os.path.join(path, c)))

num_images = sum(list(counts.values()))
print('Total number of images in dataset: {0}'.format(num_images))

# Number of images in each clsss plot
fig = plt.figure(figsize = (25, 5))
sns.barplot(x = list(counts.keys()), y = list(counts.values())).set_title('Number of images in each class')
plt.xticks(rotation = 90)
plt.margins(x=0)
plt.show()

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    dtype='float32',
    horizontal_flip = True,
    vertical_flip = True,
    rotation_range = 45,
    zoom_range = 0.2,
    width_shift_range = 0.15,
    height_shift_range = 0.15,
    shear_range = 0.2)

train_generator = train_datagen.flow_from_directory(
    'dataset/train_images',
    batch_size=8,
    target_size=(300,300),
    class_mode='sparse',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    'dataset/train_images',
    batch_size=8,
    target_size=(300,300),
    class_mode='sparse',
    subset="validation"
)

In [ ]:
train_generator.class_indices

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
model = tf.keras.applications.Xception(input_shape=(300,300,3), classes=14, weights=None)

In [ ]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model.fit_generator(train_generator, epochs=100, verbose=True, 
                    callbacks=[tensorboard_callback, checkpoint_callback, early_stop_callback],
                    validation_data=validation_generator,)

In [ ]:
scores = model.evaluate(x=validation_generator, verbose=1)